# Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm, t
from dataclasses import dataclass

@dataclass
class SimulationParams:
    n_sims : int
    n_samples : int
    beta : float
    alpha : float
    rho : float

    def __iter__(self):
        for (k, v) in self.__dict__.items():
            yield k, v

params = SimulationParams(
    n_sims = 100,
    n_samples = 100,
    beta = 0.0,
    alpha = 0.0,
    rho = 0.0,
)

it = iter(params)

res = next(it)
res = next(it)

print(res)

('n_samples', 100)


# 1) Data Generation